In [1]:
import torch, torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
tf_train = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])
tf_eval = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

In [3]:
train = torchvision.datasets.OxfordIIITPet(
    root="../data",
    split="trainval",
    target_types="binary-category",  # 0=Cat, 1=Dog
    download=True,
    transform=tf_train
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 792M/792M [00:36<00:00, 21.4MB/s]


Extracting ../data/oxford-iiit-pet/images.tar.gz to ../data/oxford-iiit-pet


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19.2M/19.2M [00:01<00:00, 10.5MB/s]


Extracting ../data/oxford-iiit-pet/annotations.tar.gz to ../data/oxford-iiit-pet


In [4]:
test = torchvision.datasets.OxfordIIITPet(
    root="../data",
    split="test",
    target_types="binary-category",
    download=True,
    transform=tf_eval
)

In [5]:
train_loader = DataLoader(train, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test,  batch_size=64, shuffle=False, num_workers=2)

In [6]:
import torch.nn as nn, torch.optim as optim

model = nn.Sequential(
    nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),   # 128->64
    nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 64->32
    nn.Conv2d(32,64,3,padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(),
    nn.Linear(64,2)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [7]:
for epoch in range(2):
    model.train()
    running = 0.0
    for x,y in train_loader:
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running += loss.item()
    print(f"Epoch {epoch+1} - Loss: {running/len(train_loader):.4f}")

Epoch 1 - Loss: 0.6429
Epoch 2 - Loss: 0.6199


In [8]:
import os, pathlib, torch
models_dir = pathlib.Path("../data/models"); models_dir.mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), models_dir/"catsdogs_model.pth")